**Install dependencies**

In [ ]:
!pip install newsapi-python pandas transformers

**Import libraries**

In [ ]:
from newsapi import NewsApiClient
import pandas as pd
from transformers import pipeline
import time
from google.colab import userdata   # for Colab Secrets

**Load API Key securely**

In [ ]:
#  Load API key from Colab Secrets
news_api_key = userdata.get('NEWS_API_KEY')

if not news_api_key:
    raise ValueError(" NEWS_API_KEY not found in Colab Secrets. Please add it first.")

# Initialize NewsAPI client
newsapi = NewsApiClient(api_key=news_api_key)


**Fetch articles**

In [ ]:
# Parameters
page_size = 100   # max articles per page
total_pages = 5   # number of pages to fetch
all_articles = []  # store articles

# Refined AI/ML query (exclude unrelated topics)
query = '"Artificial Intelligence" OR "Machine Learning" -game -gaming -sports -football -cricket'

# Fetch articles page by page
for page in range(1, total_pages + 1):
    print(f" Fetching page {page}...")
    try:
        response = newsapi.get_everything(
            q=query,
            language="en",
            page_size=page_size,
            page=page,
            sort_by="publishedAt"
        )
        all_articles.extend(response["articles"])
        time.sleep(1)  # avoid hitting API limits
    except Exception as e:
        print(f" Error fetching page {page}: {e}")
        break

print(f" Total articles fetched: {len(all_articles)}")


 Fetching page 1...
 Fetching page 2...
 Error fetching page 2: {'status': 'error', 'code': 'maximumResultsReached', 'message': 'You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 200. Please upgrade to a paid plan if you need more results.'}
 Total articles fetched: 95


**Store articles in DataFrame**

In [ ]:
# Extract relevant data
news_data = [{
    "source": a["source"]["name"],
    "text": a["title"],
    "publishedAt": a["publishedAt"]
} for a in all_articles]

df_news = pd.DataFrame(news_data)

# Show sample data
print(" Sample AI/ML news:")
print(df_news.head())


 Sample AI/ML news:
                         source  \
0                    TechCrunch   
1               Bestadsontv.com   
2  International Business Times   
3                        Forbes   
4                  BusinessLine   

                                                text           publishedAt  
0  What is Mistral AI? Everything to know about t...  2025-09-07T14:57:36Z  
1  Interactive ad: Mastercard: Mastercard Transit...  2025-09-07T14:56:41Z  
2           AI-powered Meet-up Apps Fight Loneliness  2025-09-07T14:51:17Z  
3  Crypto Is Powering The AI Age’s Most Critical ...  2025-09-07T14:40:42Z  
4  ‘For Samsung, India is the second-largest R&D ...  2025-09-07T14:37:52Z  


**Sentiment analysis**

In [ ]:
# Sentiment Analysis with Hugging Face
sentiment_pipeline = pipeline("sentiment-analysis")

# Apply to all news titles
results = sentiment_pipeline(list(df_news["text"]), batch_size=32)

df_news["sentiment"] = [r["label"] for r in results]
df_news["score"] = [r["score"] for r in results]

print("\n AI/ML news with sentiment:")
print(df_news.head())


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



 AI/ML news with sentiment:
                         source  \
0                    TechCrunch   
1               Bestadsontv.com   
2  International Business Times   
3                        Forbes   
4                  BusinessLine   

                                                text           publishedAt  \
0  What is Mistral AI? Everything to know about t...  2025-09-07T14:57:36Z   
1  Interactive ad: Mastercard: Mastercard Transit...  2025-09-07T14:56:41Z   
2           AI-powered Meet-up Apps Fight Loneliness  2025-09-07T14:51:17Z   
3  Crypto Is Powering The AI Age’s Most Critical ...  2025-09-07T14:40:42Z   
4  ‘For Samsung, India is the second-largest R&D ...  2025-09-07T14:37:52Z   

  sentiment     score  
0  NEGATIVE  0.998384  
1  POSITIVE  0.780552  
2  POSITIVE  0.974201  
3  POSITIVE  0.997610  
4  POSITIVE  0.998396  


**Save dataset**

In [ ]:
# Save to CSV
df_news.to_csv("ai_news_sentiment.csv", index=False)
print(" CSV saved successfully as 'ai_news_sentiment.csv'.")


 CSV saved successfully as 'ai_news_sentiment.csv'.
